In [1]:
#importing libraries

from tkinter import *
from PIL import ImageTk,Image
import cv2
import pyttsx3
import numpy as np
import face_recognition
import os
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import pytesseract

In [2]:
def input_new_faces():
    
    text_speech = pyttsx3.init()
    text_speech.setProperty("rate",150)
    text_speech.say("Store new faces")
    text_speech.runAndWait()
     
    def write():
        global root2
        a = entry1.get()
        a = a.lower()
        img_name = "C:/Users/DELL/OneDrive/Desktop/Face directory/{}.png".format(a)
        cv2.imwrite(img_name,img)
        
        text_speech = pyttsx3.init()
        text_speech.setProperty("rate",150)
        text_speech.say("Written")
        text_speech.runAndWait()
                    
        root2.destroy()
        root.deiconify()
        root1.deiconify()
    
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)
    voice_counter=1
    cam_counter=1
    
    while True:
        _,img = cap.read()

        if not _:
            print("failed to grab frame")
            break
            
        cv2.imshow('New face',img)
        while cam_counter<2:
            cv2.waitKey(1)
            cam_counter+=1

        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray,1.1,4)

        if len(faces) == 0:
            print("face not detected")
            text_speech = pyttsx3.init()
            text_speech.say("face not detected")
            text_speech.runAndWait()

        elif len(faces) != 0:

            for (x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

            cv2.imshow('New face',img)

            while voice_counter<2:
                    print("Face detected, press space bar to store the face")
                    text_speech = pyttsx3.init()
                    text_speech.setProperty("rate",150)
                    text_speech.say("Face detected, press space bar to store the face")
                    text_speech.runAndWait()
                    voice_counter+=1


        k = cv2.waitKey(30) & 0xff
        if k%256 == 32:
            global entry1
            global root2
            root.withdraw()
            root1.withdraw()
            root2 = Toplevel()
            root2.configure(bg = black)
            root2.geometry("360x160")
            root2.iconbitmap('AI logo.ico')
            root2.title("AI")
            root2.resizable(False,False)
            
            Label(root2, text = "Enter name ",font = ("Arial",13,"bold"),fg = blue, bg = black).place(x = 30, y = 40)
            entry1 = Entry(root2,font = ("Arial",13,"bold"))
            entry1.place(x = 130,y = 40)
            Button(root2,text = "Enter",font = ("Arial",13,"bold"),fg = "black",bg = "white",command = write).place(x = 120, y = 80)
            break

    cap.release()
    cv2.destroyAllWindows()

In [3]:
def face_recognition():
    
    import face_recognition
    import cv2

    path = r"C:\Users\DELL\OneDrive\Desktop\Face directory"
    path1 = r"C:\Users\DELL\OneDrive\Desktop\Criminal directory"
    
    text_speech = pyttsx3.init()
    text_speech.setProperty("rate",150)
    text_speech.say("Face recognition")
    text_speech.runAndWait()

    images = []
    classnames = []
    mylist = os.listdir(path)

    for cl in mylist:
        curimg = cv2.imread(f'{path}/{cl}')
        images.append(curimg)
        classnames.append(os.path.splitext(cl)[0])

    images1 = []
    classnames1 = []
    mylist1 = os.listdir(path1)

    for cl in mylist1:
        curimg = cv2.imread(f'{path1}/{cl}')
        images1.append(curimg)
        classnames1.append(os.path.splitext(cl)[0])
        
    text_speech = pyttsx3.init()
    text_speech.say("Please wait for a while")
    text_speech.runAndWait()
            
    def findencodings(images):
        encodelist = []
        for img in images:
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            encode = face_recognition.face_encodings(img)[0]
            encodelist.append(encode)
        return encodelist

    encodelistknown = findencodings(images) 
    encodelistknown1 = findencodings(images1)

    cap = cv2.VideoCapture(0)
    cam_counter = 1
    voice_counter = 1

    while True:

        k = cv2.waitKey(30) & 0xff
        if k%256 == 32:
            text_speech = pyttsx3.init()
            text_speech.setProperty("rate",160)
            
            if name.lower() in classnames1:
                print(name+" be cautious")
                text_speech.say("I can see "+name.lower()+" Be cautious")
                text_speech.runAndWait()
                break
            
            elif name.lower() in classnames:
                print(name)
                text_speech.say("I can see "+name.lower())
                text_speech.runAndWait()
                break
                
            else:
                text_speech.say("No known face detected")
                text_speech.runAndWait()
                break
                
            

        success,img = cap.read()
        imgs = cv2.resize(img,(0,0),None,0.25,0.25)
        imgs = cv2.cvtColor(imgs,cv2.COLOR_BGR2RGB)

        facescurframe = face_recognition.face_locations(imgs)
        encodescurframe = face_recognition.face_encodings(imgs,facescurframe)
        
        if len(facescurframe) == 0:
            text_speech = pyttsx3.init()
            text_speech.say("No face detected")
            text_speech.runAndWait()
            
        else:

            for encodeface,faceloc in zip(encodescurframe,facescurframe):
                matches = face_recognition.compare_faces(encodelistknown,encodeface)
                facedis = face_recognition.face_distance(encodelistknown,encodeface)
                matchindex = np.argmin(facedis)

                matches1 = face_recognition.compare_faces(encodelistknown1,encodeface)
                facedis1 = face_recognition.face_distance(encodelistknown1,encodeface)
                matchindex1 = np.argmin(facedis1)

                if matches1[matchindex1]:
                    name = classnames1[matchindex1].upper()
                
                elif matches[matchindex]:
                    name = classnames[matchindex].upper()

                else:
                    name = "UNKNOWN"

                y1,x2,y2,x1 = faceloc
                y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
                cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
                cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
                cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                
        cv2.imshow('webcam',img)
        while cam_counter<2:
                cv2.waitKey(1)
                cam_counter+=1
        while(voice_counter<2):
            text_speech = pyttsx3.init()
            text_speech.setProperty("rate",190)
            text_speech.say("Press space bar to know the person")
            text_speech.runAndWait()
            voice_counter+=1

    cap.release()
    cv2.destroyAllWindows()
    

In [4]:
def criminal_detection():
    
    text_speech = pyttsx3.init()
    text_speech.setProperty("rate",150)
    text_speech.say("Criminal directory")
    text_speech.runAndWait()
    
    def write():
        global root3
        a = entry2.get()
        a = a.lower()
        img_name = "C:/Users/DELL/OneDrive/Desktop/Criminal directory/{}.png".format(a)
        cv2.imwrite(img_name,img)
        text_speech = pyttsx3.init()
        text_speech.setProperty("rate",150)
        text_speech.say("Written")
        text_speech.runAndWait()
        root3.destroy()
        root.deiconify()
        root1.deiconify()
    
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)
    voice_counter=1
    cam_counter=1
    
    while True:
        _,img = cap.read()

        if not _:
            print("failed to grab frame")
            break
            
        cv2.imshow('New face',img)
        while cam_counter<2:
            cv2.waitKey(1)
            cam_counter+=1

        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray,1.1,4)

        if len(faces) == 0:
            print("face not detected")
            text_speech = pyttsx3.init()
            text_speech.say("face not detected")
            text_speech.runAndWait()

        elif len(faces) != 0:

            for (x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

            cv2.imshow('New face',img)

            while voice_counter<2:
                    print("Face detected, press space bar to store into criminal directory")
                    text_speech = pyttsx3.init()
                    text_speech.setProperty("rate",150)
                    text_speech.say("Face detected, press space bar to store into criminal directory")
                    text_speech.runAndWait()
                    voice_counter+=1

        k = cv2.waitKey(30) & 0xff
        if k%256 == 32:
            global entry2
            global root3
            root.withdraw()
            root1.withdraw()
            root3 = Toplevel()
            root3.configure(bg = black)
            root3.geometry("360x160")
            root3.iconbitmap('AI logo.ico')
            root3.title("AI")
            root3.resizable(False,False)
            
            Label(root3, text = "Enter name ",font = ("Arial",13,"bold"),fg = blue, bg = black).place(x = 30, y = 40)
            entry2 = Entry(root3,font = ("Arial",13,"bold"))
            entry2.place(x = 130,y = 40)
            Button(root3,text = "Enter",font = ("Arial",13,"bold"),fg = "black",bg = "white",command = write).place(x = 120, y = 80)
            break

    cap.release()
    cv2.destroyAllWindows()
    

In [5]:
def know_the_emotion():
    
    text_speech = pyttsx3.init()
    text_speech.setProperty("rate",150)
    text_speech.say("Emotion detection")
    text_speech.runAndWait()

    voice_counter=1

    while voice_counter<2:
        text_speech = pyttsx3.init()
        text_speech.say("Please wait")
        text_speech.runAndWait()
        voice_counter+=1

    face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    classifier =load_model("model.h5")

    emotion_labels = ['Angry','Disgust','Fear','Happy','Neutral', 'Sad', 'Surprise']

    cap = cv2.VideoCapture(0)
    speech_counter=1
    cam_counter=1

    while True:
        _, frame = cap.read()
        labels = []
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray)
        
        
        k = cv2.waitKey(1)
        if k%256 == 32:
            text_speech = pyttsx3.init()
            text_speech.setProperty("rate",150)
            text_speech.say("This person has "+label+" emotion")
            text_speech.runAndWait()
            break

        if len(faces) == 0:
            text_speech = pyttsx3.init()
            text_speech.setProperty("rate",150)
            text_speech.say("No face detected")
            text_speech.runAndWait()

        else:
            for (x,y,w,h) in faces:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
                roi_gray = gray[y:y+h,x:x+w]
                roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

                while speech_counter<2:
                    text_speech = pyttsx3.init()
                    text_speech.setProperty("rate",150)
                    text_speech.say("Enter space to know the emotion")
                    text_speech.runAndWait()
                    speech_counter+=1

                if np.sum([roi_gray])!=0:
                    roi = roi_gray.astype('float')/255.0
                    roi = img_to_array(roi)
                    roi = np.expand_dims(roi,axis=0)

                    prediction = classifier.predict(roi)[0]
                    label=emotion_labels[prediction.argmax()]
                    label_position = (x,y)
                    cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)


        cv2.imshow('Emotion Detector',frame)
        while cam_counter<2:
                cv2.waitKey(1)
                cam_counter+=1

    cap.release()
    cv2.destroyAllWindows()

In [6]:
def text_recognition():
    
    text_speech = pyttsx3.init()
    text_speech.setProperty("rate",150)
    text_speech.say("Text recognition")
    text_speech.runAndWait()

    cam = cv2.VideoCapture(0)
    voice_counter=1
    speech_counter=1
    cam_counter=1
    
    while True:
        ret,frame = cam.read()
        if not ret:
            print("failed to grab the frame")
            break

        cv2.imshow("text",frame)
        while cam_counter<2:
                cv2.waitKey(1)
                cam_counter+=1

        while voice_counter<2:
            text_speech = pyttsx3.init()
            text_speech.say("place the text and press space bar to read")
            text_speech.runAndWait()
            voice_counter+=1

        k = cv2.waitKey(30) & 0xff
        if k%256 == 32:
            img_name = "test.png"
            cv2.imwrite(img_name,frame)
            
            while speech_counter<2:
                text_speech = pyttsx3.init()
                text_speech.say("Recognition in process")
                text_speech.runAndWait()
                speech_counter+=1
                
            pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
            tessdata_dir_config = '--tessdata-dir "C:/Program Files/Tesseract-OCR/tessdata"'
            img = Image.open("test.png")
            text = pytesseract.image_to_string(img,config = tessdata_dir_config)
            print(text)
            if len(text) == 0:
                text_speech = pyttsx3.init()
                text_speech.setProperty("rate",150)
                text_speech.say("No text detected")
                text_speech.runAndWait()
                
            else:
                text_speech = pyttsx3.init()
                text_speech.say(text)
                text_speech.runAndWait()    
            os.remove("test.png")
            break

    cam.release()
    cv2.destroyAllWindows()
    


In [7]:
root = Tk()
home = PhotoImage(master = root,file ="AI home.png")
label = Label(root,image = home)
label.place(x = 0,y=0,relwidth = 1,relheight=1)
root.geometry("476x549")
root.iconbitmap('AI logo.ico')
root.title("AI")

blue = "#A2D2FA"
black = "#221F21"

root.configure(bg = black)
root.resizable(False,False)

def explore():
    
        global root1    
        root1 = Toplevel()
        image1 = Image.open("AI about.png")
        back_ground = ImageTk.PhotoImage(image1)
        root1.geometry("550x778")
        root1.configure(bg = black)
        root1.iconbitmap('AI logo.ico')
        root1.title('AI')
        root1.resizable(False,False)
        
        lbl = Label(root1,image = back_ground)
        lbl.place(x=0,y=0)
        b1= Button(root1,text = "Use >",font = ("Open Sans",14,"bold"),bg = black,fg = blue,width =6,command = input_new_faces).place(x=172,y=112)
        b2= Button(root1,text = "Use >",font = ("Open Sans",14,"bold"),bg = black,fg = blue,width =6,command = face_recognition).place(x=320,y=272)
        b3= Button(root1,text = "Use >",font = ("Open Sans",14,"bold"),bg = black,fg = blue,width =6,command = criminal_detection).place(x=172,y=430)
        b4= Button(root1,text = "Use >",font = ("Open Sans",14,"bold"),bg = black,fg = blue,width =6,command = know_the_emotion).place(x=320,y=572)
        b5= Button(root1,text = "Use >",font = ("Open Sans",14,"bold"),bg = black,fg = blue,width =6,command = text_recognition).place(x=177,y=717)

        root1.mainloop()
        

b = Button(root,text = "Explore >",font = ("Open Sans",20,"bold"),width =10,bg = "white",fg = "black",command = explore).place(x=100,y=255)

root.mainloop()

Face detected, press space bar to store the face
face not detected
face not detected
face not detected
ANANYA
Face detected, press space bar to store into criminal directory
ANANYA be cautious
They said, don’t give up on your dreams, So, | went back to.
sleep

Cut the trees, make paper with them and write “Save the
trees”.

